# Шифрование данных

Необходимо разработать метод преобразования данных для модели линейной регрессии, чтобы сложно было восстановить персональную информацию. При этом качество модели машинного обучения не должно ухудшиться.

### Описание данных

- признаки: пол, возраст и зарплата застрахованного, количество членов его семьи;
- целевой признак: количество страховых выплат клиенту за последние 5 лет.

### Содержание

1. Изучение общей информации о данных
2. Умножением матриц
3. Алгоритм преобразования
4. Проверка алгоритма

### Библиотеки и модули

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Загрузка данных

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv')

In [3]:
insurance.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
X = insurance.drop(['Страховые выплаты'], axis=1)
y = insurance['Страховые выплаты']

## Умножение матриц

**Ответ:** 
1. Изменится качество в худшую сторону, если сама матрица $X_{m\times n}$ является обратимой и мы умножаем ее на обратную ей $X_{n\times m}^{-1}$.
2. Качество не изменится, если умножим матрицу $X_{m\times n}$ на единичную $E_{n\times n}$ (единичная матрица обратимая).
3. Умножим матрицу $X_{m\times n}$ на матрицу $P_{n\times n}$, которая является является обратимой.  

**Обоснование:** 
1. При умножении матрицы $X$ на обратную ей получим единичную: $XX^{-1}=X^{-1}X=E$. Единичная матрица не отражает данные, а лишь является матрицей, заполненной нулями и единицами по главной диагонали.
2. При умножении матрицы $X$ на единичную получим ту же самую матрицу: $XE=EX=X$.
3. Тогда получим следующую формулу весов: $w'=((XP)^T(XP))^{-1}(XP)^Ty$. 

$$
[свойства: (AB)^T=B^TA^T, (AB)^{-1}=B^{-1}A^{-1}] 
$$
$$
w=((XP)^T(XP))^{-1}(XP)^Ty=
$$
$$
=P^{-1}(XX^T)^{-1}(P^T)^{-1}P^TX^Ty=
$$
$$
=P^{-1}(XX^T)^{-1}EX^Ty=
$$
$$
=P^{-1}(XX^T)^{-1}X^Ty
$$ 
Передадим вектор признаков в алгоритм модели: $a=Xw$. 

$$
a=Xw'=(XP)w'=XPP^{-1}(X^TX)^{-1}X^Ty=XEw=Xw, \text{где } w=(X^TX)^{-1}X^Ty \text{ - вектор весов без умножения на матрицу $P$}.
$$ 

Получаем, что умножение матрицы $X$ на матрицу $P$ не изменяет качество модели $a$.

## Алгоритм преобразования

1. Создадим матрицу $P_{n\times n}$, заполненную случайными значениями из нормального распределения.
2. Сделаем проверку на обратимость матрицы $P$.  
    1.1. Если матрица обратимая, то умножаем ее на матрицу $X$, далее обучаем модель на преобразованных признаках.  
    1.2. В другом случае генерируем новую матрицу до тех пор, пока матрица не станет обратимой.

## Проверка алгоритма

In [6]:
def X_new(X):
    try:
        P = np.random.normal(size=(X.shape[1], X.shape[1]))
        P_inv = np.linalg.inv(P)
        return X @ P
    except:
        X_new(X)

In [7]:
X_new = X_new(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [9]:
X_train_new, X_test_new, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=7)

### До преобразования

In [10]:
model_1 = LinearRegression()
model_1.fit(X_train, y_train)
y_pred = model_1.predict(X_test)
print(f'Качество модели по метрике R2 до преобразования: {r2_score(y_test, y_pred)}')

Качество модели по метрике R2 до преобразования: 0.4059692352672236


### После преобразования

In [11]:
model_2 = LinearRegression()
model_2.fit(X_train_new, y_train)
y_pred = model_2.predict(X_test_new)
print(f'Качество модели по метрике R2 после преобразования: {r2_score(y_test, y_pred)}')

Качество модели по метрике R2 после преобразования: 0.4059692352672526
